<h1> Part 1(a): Employee with greater salary than their manager </h1>

In [541]:
import pandas as pd

In [542]:
empl_df = pd.read_csv("data/employee_test.csv")
empl_df.head()

,id,name,salary,manager_id
0,1,John,300,3.0
1,2,Mike,200,3.0
2,3,Sally,550,4.0
3,4,Jane,500,7.0
4,5,Joe,600,7.0


In [543]:
def empl_greater_salary(empl_df):
    """
    Retrieve list of employees with salary grater than their manager

    Arguments:
    empl_df -- employee information, dataframe with 4 columns

    Return:
    greater_empl_names -- list of employees, list
    empl_mgr_df -- employee and their respective manager's information, dataframe with 6 columns 
    """

    # Create a new table with both employee and manager information
    empl_mgr_df = empl_df.merge(empl_df[['id', 'salary']],
                        how='left',
                        left_on='manager_id',
                        right_on='id',
                        suffixes=('_empl', '_mgr'))
    
    # Apply filter where an employee's salary is larger than their immediate manager
    greater_empl_df = empl_mgr_df[empl_mgr_df['salary_empl'] > empl_mgr_df['salary_mgr']]

    # Retrieve the names of employees fulfilling the above filter
    greater_empl_names = greater_empl_df['name'].to_list()

    # Sanity assertions
    assert empl_mgr_df.shape[0] == empl_df.shape[0], "Mismatched in DataFrame sizes."

    return greater_empl_names, empl_mgr_df


In [544]:
greater_empl_names, _ = empl_greater_salary(empl_df)
print("People with salaries greater than their immediate manager:", 
        ", ".join(greater_empl_names))

People with salaries greater than their immediate manager: Sally, Joe, Dan


<h1> Part 1(b): Average salary of non-managers </h1>

In [546]:
def non_mgr_salary(empl_df):
    """
    Compute the average salary of non-managers

    Arguments:
    empl_df -- employee information, dataframe with 4 columns

    Return:
    avg_non_mgr_salary -- average salary of non-managers, float 2 decimals
    non_mgr_df -- information of non-manager employee, dataframe with 4 columns 
    """

    # Get a list of employee IDs who are managers
    mgr_list = empl_df['manager_id'].dropna().unique()

    # Apply filter where employee ID is not in managers list
    non_mgr_df = empl_df[~empl_df['id'].isin(mgr_list)]

    # Take the average of the non-managers salary
    avg_non_mgr_salary = non_mgr_df['salary'].mean()

    return round(avg_non_mgr_salary, 2)

In [547]:
avg_non_mgr_salary = non_mgr_salary(empl_df)
print("Average salary of employees who do not manage anyone:", avg_non_mgr_salary)

Average salary of employees who do not manage anyone: 425.0


<h1> Part 2: Exists </h1>

In [8]:
def exists(var):
    """
    Check if a variable symbol exists globally

    Arguments:
    var -- symbol of variable, string

    Return:
    boolean variable whether var exist in globals
    """
    if type(var) != str:
        raise TypeError("Input has to be a string.")
    
    return var in globals()

In [9]:
a = 1
print("Does variable 'a' exist?", exists('a'))
print("Does variable 'b' exist?", exists('b'))

Does variable 'a' exist? True
Does variable 'b' exist? False


In [10]:
print("Does variable 'a' exist?", exists(a))

TypeError: Input has to be a string.

<h1>Part 3: Pascal triangle</h1>


For an element $i$ in row $n$ in a Pascal triangle:
$$
C_{n, i} = \frac{n!}{i!(n-i)!}\\
$$

To simplify calculation, derive the next term in relation to the previous term:
$$
C_{n, i+1} = \frac{n!}{(i+1)!(n-(i+1))!}\\
C_{n, i+1} = \frac{n!}{i!(n-i)!}\frac{(n-i)}{(i+1)}\\
C_{n, i+1} = C_{n, i}\frac{(n-i)}{(i+1)}\\
$$


In [203]:
def pascal_row(n):
    """
    Compute the n-th row of the Pascal triangle

    Arguments:
    n -- row index, integer

    Return:
    row -- n-th row of the Pascal triangle, list
    """
    if type(n) != int:
        raise TypeError("Input has to be an integer.")
    elif n < 0:
        raise ValueError("n cannot be negative.")

    row = [1]
    for i in range(n-1):
        term = int(row[i]*(n - i)/(i + 1))
        row.append(term)

    return row

In [199]:
row = pascal_row(7)
print("7-th row of Pascal triangle:", " ".join([str(i) for i in row]))

7-th row of Pascal triangle: 1 7 21 35 35 21 7


<h1>Part 4(a): VaR95% and CVaR95% using historical daily returns</h1>

Assumptions:
- Buy at close on 2016/01/04 (for symmetry)
- Daily returns = (prev. day's closing - today's closing)/prev. day's closing
- There are no dividends
- Trading days are continuous (weekends and off days are ignored)
- Stocks are divisible e.g. we can own 0.9 of an AAPL stock
- Assume no rebalancing

In [368]:
# !pip install yfinance
import yfinance as yf
import numpy as np

In [462]:
allocation = {
    'AAPL': 0.15,
    'IBM': 0.20,
    'GOOG': 0.20,
    'BP': 0.15,
    'XOM': 0.10,
    'COST': 0.15,
    'GS': 0.05
}

tickers = list(allocation.keys())
weights = np.array(list(allocation.values()))

assert weights.sum() == 1, "Weights do not sum up to 1"

In [463]:
history = yf.download(tickers, start = '2016-01-01', end = '2016-12-31')
history = history["Close"]

[*********************100%***********************]  7 of 7 completed


In [465]:
def var_historical(history, weights, alpha):
    """
    Compute historical VaR and CVaR for a given data

    Arguments:
    history -- historical price data (output from yahoo finance download), dataframe
    weights -- list of weights (same ticker order as history columns), np.array
    alpha -- level of significance, int

    Return:
    var -- float
    cvar -- float
    """
    if alpha > 1 or alpha < 0:
        raise ValueError("Alpha must be between 0 and 1")

    # Compute daily returns
    daily_returns = history.pct_change()[1:]

    # Compute cumulative daily returns
    cum_returns = (1 + daily_returns).cumprod()

    # Multiply cumulative returns with weights to calculate total portfolio value
    pfl_value = (cum_returns * weights).sum(axis=1)
                    
    # Compute portfolio daily returns                
    pfl_returns = pfl_value.pct_change()[1:]

    # VaR is equal to the a-th quantile
    var = pfl_returns.quantile(alpha)

    # CVaR is equal to the mean of returns below VaR
    cvar = pfl_returns[pfl_returns <= var].mean()

    return var, cvar 

In [466]:
var95, cvar95 = var_historical(history, weights, alpha=0.05)
print("Historical VaR95% = {:.2f}%, Historical CVaR95% = {:.2f}%".format(var95*100, cvar95*100))

Historical VaR95% = -1.47%, Historical CVaR95% = -2.22%


<h1>Part 4(b): VaR95% and CVaR95% using expected mean, covariance matrix and parametric method</h1>

In [467]:
from scipy.stats import norm

In [471]:
def var_parametric(history, weights, alpha):
    """
    Compute VaR and CVaR for a given data using parametric method

    Arguments:
    history -- historical price data (output from yahoo finance download), dataframe
    weights -- list of weights (same ticker order as history columns), np.array
    alpha -- level of significance, int

    Return:
    var -- float
    cvar -- float
    """
    if alpha > 1 or alpha < 0:
        raise ValueError("Alpha must be between 0 and 1")

    # Compute daily returns
    daily_returns = history.pct_change()[1:]

    # Compute covariance matrix
    cov_matrix = daily_returns.cov()

    # Compute expected returns for each ticker
    avg_returns = daily_returns.mean()

    # Compute portfolio daily returns mean and standard deviation
    pfl_ret_mean = avg_returns.dot(weights)
    pfl_ret_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))

    # VaR is equal the a-th percentile assuming normal distribution
    var = norm.ppf(alpha)*pfl_ret_stdev + pfl_ret_mean

    # CVaR is equal to the expectation of returns below VaR
    cvar = pfl_ret_mean - (1/alpha)*norm.pdf(norm.ppf(alpha))*pfl_ret_stdev

    return var, cvar

In [472]:
var95, cvar95 = var_parametric(history, weights, alpha=0.05)
print("Parametric VaR95% = {:.2f}%, Parametric CVaR95% = {:.2f}%".format(var95*100, cvar95*100))

Parametric VaR95% = -1.49%, Parametric CVaR95% = -1.89%


<h1>Part 4(c): Optimal portfolio</h1>

Assumptions:
- for each month rebalancing, we use all available historical data from 2016/01/01 onwards to optimize portfolio
- assume 252 trading days, 21 days a month
- optimal portfolio has the highest Sharpe ratio
- monthly risk free rate $\approx$ 0.12%

In [525]:
def sim_portfolios(history, n=10000):
    """
    Run a simulation of portfolios with randomized weights

    Arguments:
    history -- historical price data (output from yahoo finance download), dataframe
    n -- number of iterations, integer

    Return:
    portfolios -- simulation results containing weights, returns and volatility, dataframe
    """
    tickers = list(history.columns)
    portfolios = [] # Empty list to store simulation results

    # Compute daily returns
    daily_returns = history.pct_change()[1:]

    # Compute covariance matrix
    cov_matrix = daily_returns.cov()

    # Compute expected returns for each ticker for 1-month's horizon
    avg_returns = history.resample('M').last().pct_change().mean()

    # Simulate n portfolios with random weights
    for i in range(n):
        weights = np.random.uniform(-1, 1, len(tickers)) # Initialize random weights
        weights = weights/np.sum(weights) # Normalize weights to a sum of 1

        # Skip iteration if normalized weights have values bigger than 1
        if sum(abs(weights) > 1) > 0:
            continue

        returns = avg_returns.dot(weights) # Compute expected portfolio returns
        
        # Compute daily volatility
        volatility = np.sqrt(weights.T.dot(cov_matrix).dot(weights))*np.sqrt(21)

        # Store result in list
        portfolios.append(weights.tolist() + [returns, volatility])

    # Store results in a dataframe
    portfolios = pd.DataFrame(portfolios)
    portfolios.columns = tickers + ["Returns", "Volatility"]

    return portfolios

In [534]:
def monthly_rebalance(history, n=10000, rf=0.0012):
    """
    Rebalance portfolio at the end of every month using simulation

    Arguments:
    history -- historical price data (output from yahoo finance download), dataframe
    n -- number of iterations, integer

    Return:
    weights -- rebalanced weights at the end of every month, dataframe
    """
    tickers = list(history.columns)
    
    # Get a list of end-of-month dates
    eom = history.resample('M').last()

    weights = []

    for date in eom.index[1:]:
        # Run a simulation for the given end-of-month
        portfolios = sim_portfolios(history[:date], n=n)

        # Get the index of the portfolio with the highest Sharpe ratio
        optimal_pfl_index = ((portfolios['Returns'] - rf)/portfolios['Volatility']).idxmax()

        # Get the weights, returns and volatility details of the optimal portfolio
        optimal_pfl = portfolios.iloc[optimal_pfl_index]

        # Add optimal portfolio details in weights
        weights.append(optimal_pfl[:len(tickers)].to_list())

    # Store optimal portfolio details in a dataframe
    weights = pd.DataFrame(weights, index=eom.index[1:])
    weights.columns = tickers

    return weights

In [535]:
monthly_weights = monthly_rebalance(history, n=10000)
print("Optimal portfolio holding by end of each month:")
monthly_weights

Optimal portfolio holding by end of each month:


,AAPL,BP,COST,GOOG,GS,IBM,XOM
Date,,,,,,,
2016-01-31,-0.137871,0.938628,0.193286,0.623872,-0.918836,-0.353434,0.654355
2016-02-29,0.391862,-0.273992,-0.511220,0.397057,-0.821566,0.874927,0.942933
2016-03-31,0.264058,-0.147955,-0.126489,0.364385,-0.820371,0.890063,0.576309
2016-04-30,-0.277878,0.536469,0.038647,-0.046260,-0.916045,0.722987,0.942078
2016-05-31,0.352262,-0.088291,-0.652819,0.346770,-0.418369,0.697990,0.762458
2016-06-30,-0.422131,0.447595,0.429612,-0.163247,-0.711173,0.711924,0.707421
2016-07-31,0.117086,-0.065367,0.102645,0.052224,-0.709236,0.802340,0.700308
2016-08-31,0.054657,-0.145198,0.208894,0.013611,-0.432460,0.787875,0.512622
2016-09-30,0.268897,-0.086187,0.019806,0.138269,-0.839019,0.953956,0.544277


<h1>Part 5(a): Count of Python Files</h1>

Using Linux commands.

In [28]:
%%bash

find ./data/my-python-project -type f -name "*.py" | wc -l
#type f: files only
#name "*.py": name must have .py extension
#wc -l: count the number of lines of the output

      10


<h1>Part 5(b): Count number of lines excluding comment and empty lines</h1>

In [24]:
%%bash

cat `find ./data/my-python-project -type f -name "*.py" ` | tr -d " \t" | grep . | grep -v ^# | wc -l
#cat: list of contents
#tr -d: trim tab and empty spaces
#grep .: remove empty lines (. matches with all characters except return)
#grep -v ^#: retrieve non-matching lines i.e. lines not starting with #

     126


<h1>Part 5(c): Count of defined functions</h1>

In [49]:
%%bash

cat `find ./data/my-python-project -type f -name "*.py" ` | tr -d " \t" | grep . | grep ^def | wc -l
#grep ^def: retrieve matching lines i.e. lines starting with def

       7


<h1>Part 5(d): Count number of changed lines</h1>

In [565]:
%%bash

!git --git-dir ./data/my-python-project/.git diff --shortstat HEAD~3 HEAD
#--git-dir: change git directory to run command on
#diff: changelogs
#--shortstat: get only the summary of insertions and deletions

 3 files changed, 13 insertions(+), 7 deletions(-)


<h1>Part 5(e): Total folder size in MB per each of the subfolder</h1>

In [2]:
%%bash

du -h -d 2 ./data/ | awk '{printf "%.4f MB %s\n", $1/1024, $2}'

#-h: readable format
#-d: max-depth
#awk: perform action on each line


0.0000 MB ./data//my-python-project/sub-folder2
0.0234 MB ./data//my-python-project/sub-folder1
0.0742 MB ./data//my-python-project/.git
0.1094 MB ./data//my-python-project
0.1250 MB ./data/


<h1>Part 6: Count Dates</h1>

In [537]:
import re
import dateutil

In [538]:
# Formats for date parts
DAY = "\d{2}" # Capture any 2 digits number
MONTH = "\d{2}" # Capture any 2 digits number 
MONTHSTR = "Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec" # Capture abbreviated months
YEAR = "\d{4}" # Capture any 4 digits number 

DATE_PATTERNS = r"""
    (?:{year})/(?:{month})/(?:{day}) | # Capture YYYY/MM/DD format
    (?:{month})/(?:{day})/(?:{year}) | # Capture MM/DD/YYYY format
    (?:{day})/(?:{month})/(?:{year}) | # Capture DD/MM/YYYY format
    (?:{day})[ ](?:{monthstr})[ ](?:{year}) # Capture DD MMM YYYY format
"""

DATE_PATTERNS = DATE_PATTERNS.format(
    day=DAY,
    month=MONTH,
    monthstr=MONTHSTR,
    year=YEAR
)

DATE_REGEX = re.compile(DATE_PATTERNS, re.VERBOSE) # Compile with verbose

In [539]:
def count_dates(text):
    """
    Count the number of date occurences in a text

    Arguments:
    text -- string

    Return:
    count -- number of date occurences, integer
    """
    # Initiate count to zero
    count = 0
    
    # Loop through all potential matches
    for val in DATE_REGEX.findall(text):
        try: 
             # Pass through potential match into a built-in date parse to check validity
            dateutil.parser.parse(val)
            count += 1
        except ValueError:
            pass

    return count

In [540]:
with open('data/date_text.txt') as f:
    text = f.read()

count = count_dates(text)
print("Number of date occurences: ", count)

Number of date occurences:  8
